# Introducción a Grandes Modelos de Lenguaje
## Introducción a las Ciencias Sociales Computacionales
## Facultad de Ciencias Sociales, Universidad de Buenos Aires, 2023
### Juan Manuel Pérez y Rodolfo Elbert


En esta notebook vamos a jugar un poco con la API de OpenAI.

Regístrense en [la página de OpenAI y generen una API KEY](https://platform.openai.com/account/api-keys)

(No vamos a gastar mucha plata, se los juro)

Una vez que la generen, la ponen acá (no la copypasteo por obvias razones)

In [2]:
import openai

API_KEY = input("Enter your API key: ")



openai.api_key = API_KEY

## Zero-shot classification

Podemos usar a ChatGPT para que haga tareas para las cuales no fue entrenado. Por ejemplo, podemos usarlo para clasificar textos.

Para eso, usaremos un prompt que le diga a ChatGPT qué hacer. Por ejemplo, si le decimos "Clasificá este texto como positivo o negativo", y le damos un texto, nos va a decir si es positivo o negativo.

In [3]:
# Load gpt-3.5 turbo model
from openai import ChatCompletion

text = """Clasificar el siguiente texto en positivo, negativo, o neutral.

MESSI SOS EL GOAT LPM
"""


ChatCompletion.create(
    messages=[
        {"role": "system", "content": text},
    ],
    model="gpt-3.5-turbo"
)

<OpenAIObject chat.completion id=chatcmpl-8Gf3BF6JvqCppEnGD86L0Jk5MKUdL at 0x7f4e0a110770> JSON: {
  "id": "chatcmpl-8Gf3BF6JvqCppEnGD86L0Jk5MKUdL",
  "object": "chat.completion",
  "created": 1698981737,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "positivo"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 32,
    "completion_tokens": 2,
    "total_tokens": 34
  }
}

In [4]:

text = """Listar las entidades nombradas del texto. Consideramos: organizaciones, personas, lugares, fechas, productos, etc.

Ayer fui a ver lo que el viento se llevó con unos amigos al cine de Belgrano
"""


resp = ChatCompletion.create(
    messages=[
        {"role": "system", "content": text},
    ],
    model="gpt-3.5-turbo"
)

print(resp)

{
  "id": "chatcmpl-8Gf3IX7TMrBFxxxjsH7wCWYQjv586",
  "object": "chat.completion",
  "created": 1698981744,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "- Cine de Belgrano (lugar)\n- Ayer (fecha)\n- Lo que el viento se llev\u00f3 (producto)"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 59,
    "completion_tokens": 29,
    "total_tokens": 88
  }
}


## Few-shot learning

Acá "entrenamos" a ChatGPT para una tarea explicándole qué tipo de salidas esperamos, y mostrándole algunos ejemplos. La literatura indica que con un par de ejemplos la performance mejora sensiblemente. 

In [69]:

prompt = """Detectar si existe discurso de odio contra un grupo de personas en el siguiente texto. Listar el grupo atacado. Si no hay grupo atacado, responder "nada". Si hay insulto o agresión, responder "insulto".
---
texto: Estoy cansado de tanto planes para los parásitos que no trabajan
respuesta: clasismo
---
texto: Los judíos son una raza inferior y deberían ser exterminados
respuesta: racismo, antisemitismo
---
texto: Naaaaaaah sos una hdrmp
respuesta: insulto
---
texto: vamo bokita carajo
respuesta: nada
---
texto: El feminismo es una corriente de pensamiento que busca la igualdad de derechos entre hombres y mujeres
respuesta: nada
---
texto: Cárcel o bala
respuesta:"""

resp = ChatCompletion.create(
    messages=[{"role": "system", "content": prompt}],
    model="gpt-3.5-turbo"
)

resp

<OpenAIObject chat.completion id=chatcmpl-8GgMqIc6haW72iONwcmQhk0YKtESx at 0x7f4dc690e330> JSON: {
  "id": "chatcmpl-8GgMqIc6haW72iONwcmQhk0YKtESx",
  "object": "chat.completion",
  "created": 1698986800,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "insulto"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 190,
    "completion_tokens": 3,
    "total_tokens": 193
  }
}

## Tarea ad hoc

Vamos a pedirle que nos resalte conceptos interesantes de las encuestas de percepción de clase.


In [33]:
import pandas as pd

pd.options.display.max_colwidth = 1000

df = pd.read_csv("https://raw.githubusercontent.com/finiteautomata/csc-2023/main/data/datos_clase.csv")
# Lo shuffleamos para que no esté ordenado por clase
df.columns = ["id", "identidad autopercibida", "texto"]

df = df.sample(frac=1)

df.sample(10)


,id,identidad autopercibida,texto
285,782,Media,"PORQUE LLEGUE A LA CLASE ESTUDIANDO, TRABAJANDO"
742,719,Obrera,porque se ganó la vida trabajando
102,171,Media,la única distinción es subjetiva sobre el ingreso
510,30,Media,"se establece de acuerdo a los ingresos, que en mi casa no los genero yo, pero bueno"
590,979,Obrera,A LA CLASE TRABAJADORA. FORMO PARTE DEL GRUPO QUE SALE A BUSCAR TRABAJO
670,733,Obrera,"porque es un trabajador, saliendo adelante por sus propios medios"
710,929,Obrera,porque nací obrera.porque no soy la hermana de macri
192,368,Media,"por la posición, tengo mi casa, buenos trabajos, somos independientes"
221,764,Media,POR MI NIVEL ECONÓMICO
782,463,Obrera,porque son laburantes


In [34]:

df.sample(10)

,id,identidad autopercibida,texto
295,931,Media,PORQUE ME MANEJO CON MI TRABAJO NO SOY RICO PERO TAMPOCO POBRE
835,1134,Obrera,PORQUE TRABAJABAN PARA TENER UN MINIMO DE VIDA DIGNA
906,396,Obrera,si no trabaja no come
603,647,Obrera,hace esfuerzos para vivir
516,107,Media,status de vida
349,892,Media,"porque no vivo en un barrio muy humilde, puedo pagar mis impuestos"
649,340,Obrera,POR LO QUE GANO
7,417,Baja,"NO TRABAJO, TENGUNA UNA PENSION MINIMA Y CON ESO VIVO"
489,219,Media,porque viene de familia trabajadora (obrera) a través de los padres pudo acceder a su educación98
971,492,Obrera,trabajó toda su vida


In [45]:

prompt = """Resaltar conceptos mencionados en estas respuestas de encuestas, relacionados a la pertenencia de clase de una persona. Por ejemplo, trabajo, familia, ingresos, vivienda, cultura, educación.
---
texto: porque trabajó toda su vida, nadie le regaló nada
respuesta: trabajo
---
texto: porque tengo necesidades básicas cubiertas, acceso educativas comida
respuesta: educación, comida
---
texto: por una cuestión de ingresos y calidad de vida a nivel de consumo
respuesta: ingresos, consumo
---
texto: porque tengo un auto, una casa, un trabajo, una familia
respuesta: trabajo, familia, vivienda

---
texto: si no trabajo no como
respuesta: trabajo, comida
---
texto: PORQUE SOY LABURANTE
respuesta: trabajo
---
texto: {texto}
respuesta:
"""

#resp = ChatCompletion.create(
#    messages=[{"role": "system", "content": prompt}],
#    model="gpt-3.5-turbo"
#)
#
#resp

In [46]:

chatgpt_response = []

In [51]:
from tqdm.auto import tqdm

prev_len = len(chatgpt_response)

for i, (idx, row) in tqdm(enumerate(df.iterrows()), total=len(df)):
    if i < prev_len:
        # Esto es para que no se repita -- tuve que reiniciar un par de veces
        continue
    texto = row["texto"]
    formatted_prompt = prompt.format(texto=texto)

    resp = ChatCompletion.create(
        messages=[{"role": "system", "content": formatted_prompt}],
        model="gpt-3.5-turbo"
    )
    chatgpt_response.append(resp)
    #print("---")
    #print(texto)
    #print(chat_gpt_resp)

  0%|          | 0/983 [00:00<?, ?it/s]

In [52]:
# Me quedo con el subset que pudo responder


df["chatgpt_response"] = chatgpt_response


In [53]:
responses = [x["choices"][0]["message"]["content"] for x in chatgpt_response]

In [55]:
df["predicciones_chatgpt"] = responses

In [61]:
df.sample(10)

,id,identidad autopercibida,texto,chatgpt_response,predicciones_chatgpt
559,6,Media alta,ingresos,"{'id': 'chatcmpl-8Gg7pVo4RhI8oa0iTN3g4sV0EFCie', 'object': 'chat.completion', 'created': 1698985869, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""ingresos"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 191, 'completion_tokens': 3, 'total_tokens': 194}}",ingresos
536,283,Media,"toda la vida laburé, pero me doy unos lujos","{'id': 'chatcmpl-8Gfw4B6Rav9e8IVSBXmr3Tho3EFzR', 'object': 'chat.completion', 'created': 1698985140, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""trabajo, consumo"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 204, 'completion_tokens': 4, 'total_tokens': 208}}","trabajo, consumo"
517,117,Media,tenemos las necesidades básicas satisfechas y trabajamos los dos,"{'id': 'chatcmpl-8Gg72VI7nTcGo4tDfTNoEcva2ytUI', 'object': 'chat.completion', 'created': 1698985820, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""trabajo"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 204, 'completion_tokens': 2, 'total_tokens': 206}}",trabajo
57,402,Baja,PORQUE SIEMPRE NOS COSTO SALIR ADELANTE,"{'id': 'chatcmpl-8GgEbJOZsfDF9vMxUu1pcRe97DpkS', 'object': 'chat.completion', 'created': 1698986289, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""trabajo"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 204, 'completion_tokens': 2, 'total_tokens': 206}}",trabajo
85,113,Media,"clase media vivo en una casa mas o menos confortable y no soy de la alta porque no puedo quizás darme algunos lujos no tengo auto, no tengo casa propia","{'id': 'chatcmpl-8Gg6WwDUaWLsg9orKfM0vuLQg25et', 'object': 'chat.completion', 'created': 1698985788, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""clase media, vivienda, lujos, auto, casa propia"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 225, 'completion_tokens': 15, 'total_tokens': 240}}","clase media, vivienda, lujos, auto, casa propia"
232,876,Media,"POR NIVEL DE VIDA, INGRESOS","{'id': 'chatcmpl-8Gg6cKf2EzM9rRbWW0ZfcCEMrOL9W', 'object': 'chat.completion', 'created': 1698985794, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""ingresos, nivel de vida"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 200, 'completion_tokens': 7, 'total_tokens': 207}}","ingresos, nivel de vida"
860,912,Obrera,porque trabajó toda su vida,"{'id': 'chatcmpl-8Gg7bXfUmDzzGeMomDm7JruS1Utj8', 'object': 'chat.completion', 'created': 1698985855, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""trabajo"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 195, 'completion_tokens': 2, 'total_tokens': 197}}",trabajo
31,743,Baja,PORQUE NO ALCANZA EL SUELDO,"{'id': 'chatcmpl-8Gg55CtpveoSeSqmhDj2smLBkRuRI', 'object': 'chat.completion', 'created': 1698985699, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""ingresos"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 200, 'completion_tokens': 3, 'total_tokens': 203}}",ingresos
285,782,Media,"PORQUE LLEGUE A LA CLASE ESTUDIANDO, TRABAJANDO","{'id': 'chatcmpl-8Gg8fe9r4F8hMy8JrbGv9y60YRslb', 'object': 'chat.completion', 'created': 1698985921, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': { ""role"": ""assistant"", ""content"": ""estudio, trabajo"" }, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 209, 'completion_tokens': 4, 'total_tokens': 213}}","estudio, trabajo"
640,629,Obrera,POR EL TRABAJO,"{'id': 'chatcmpl-8Gg6knAMwzMmeoyOp75Ovba5DR9so', 'object': 'chat.completion', 'created': 1698985802, 'model': 'gpt-3.5-

In [63]:
del df["chatgpt_response"]

In [64]:
df.to_csv("datos_clase_chatgpt.csv", index=False)

In [32]:

df_subset["identidad autopercibida"].value_counts()

Media    349
Baja      74
Name: identidad autopercibida, dtype: int64

## Ejercicio

1. Intentar predecir la clase del sujeto en base a su descripción. 